In [2]:
import os
import torch
import torchvision

os.makedirs("modules")

>### data_preprocessing.py

In [4]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int = NUM_WORKERS
):
    train_data = datasets.ImageFolder(
        root=train_dir,
        transform=transform,
        target_transform=None
    )
    
    test_data = datasets.ImageFolder(
        root=train_dir,
        transform=transform,
        target_transform=None
    )
    
    train_dataloader = DataLoader(
        dataset=train_data,
        batch_size=batch_size,
        num_workers=NUM_WORKERS,
        pin_memory=True,
        shuffle=True
        )
    
    test_dataloader = DataLoader(
        dataset=test_data,
        batch_size=batch_size,
        pin_memory=True,
        num_workers=NUM_WORKERS,
        shuffle=False)
    
    class_names = train_data.classes
    
    return train_dataloader, test_dataloader, class_names

In [11]:
from pathlib import Path
train_dir = Path("data/pizza_steak_sushi/train")
test_dir = Path("data/pizza_steak_sushi/test")

train_dir, test_dir

(PosixPath('data/pizza_steak_sushi/train'),
 PosixPath('data/pizza_steak_sushi/test'))

In [13]:
data_transform = transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.ToTensor
])

In [15]:
train_dataloader, test_dataloader = create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               transform=data_transform,
                                                                               batch_size=32
                                                                               )